# Exercises

## 1. Sensor data exercises
In the file “data/sensors/sensor-sample.txt” you will find on each line, multiple fields of information, let’s call them : Date(Date), Time(Time), RoomId(Integer)-SensorId(Integer), Value1(float), Value2(float)
Using this file, use spark to compute the following queries :

1. Count the number of entries for each day.
2. Count the number of measures for each pair of RoomId-SensorId.
3. Compute the average of Value1.

In [39]:
from pyspark import SparkConf
# Starting spark and creating a new session
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass

config = SparkConf().setAppName('sensor')\
                    .setMaster('local[*]')

In [40]:
from pyspark import SparkContext
sc = SparkContext(conf=config)

In [41]:
sc._conf.getAll()

[('spark.app.id', 'local-1606425401953'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '41433'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'sensor'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '0f940969f428')]

In [4]:
filename = 'sensor-sample.txt'
sensorRDD = sc.textFile(filename)

In [5]:
# viewing first few elements of the RDD
sensorRDD.take(10)

['2017-03-31 03:38:16.508 1-0 122.153 2.03397',
 '2017-03-31 03:38:15.967 1-1 -3.91901 2.09397',
 '2017-03-31 03:38:16.577 1-2 11.04 2.07397',
 '2017-02-28 00:59:16.359 1-0 19.9884 2.74964',
 '2017-02-28 00:59:16.803 1-1 37.0933 2.76964',
 '2017-02-28 00:59:16.526 1-2 45.08 2.77964',
 '2017-02-28 01:03:17.244 1-0 19.3024 2.72742',
 '2017-02-28 01:03:16.972 1-1 38.4629 2.74742',
 '2017-02-28 01:03:16.914 1-2 45.08 2.77742',
 '2017-02-28 01:06:17.010 1-0 19.1652 2.70742']

### Exercise 1: Count the number of entries for each day.

In [6]:
#Splitting each line and extracting only the date column
day_RDD = sensorRDD.map(lambda line: line.split()[0])
day_RDD.take(10)

['2017-03-31',
 '2017-03-31',
 '2017-03-31',
 '2017-02-28',
 '2017-02-28',
 '2017-02-28',
 '2017-02-28',
 '2017-02-28',
 '2017-02-28',
 '2017-02-28']

In [7]:
#count number how many times each date appears in day_RDD
day_RDD.countByValue()

defaultdict(int,
            {'2017-03-31': 3393,
             '2017-02-28': 62103,
             '2017-03-01': 33423,
             '2017-03-02': 32403,
             '2017-03-03': 29727,
             '2017-03-04': 30225,
             '2017-03-05': 26019,
             '2017-03-06': 24315,
             '2017-03-07': 26625,
             '2017-03-08': 29343,
             '2017-03-09': 27288,
             '2017-03-21': 19410,
             '2017-03-22': 10989,
             '2017-03-10': 12483,
             '2017-03-23': 24213,
             '2017-03-24': 13467,
             '2017-03-11': 19059,
             '2017-03-12': 25089,
             '2017-03-25': 12225,
             '2017-03-13': 24783,
             '2017-03-26': 13587,
             '2017-03-14': 23418,
             '2017-03-27': 14544,
             '2017-03-15': 11901,
             '2017-03-28': 22338,
             '2017-03-29': 12120,
             '2017-03-16': 13869,
             '2017-03-17': 26922,
             '2017-03-30': 5814,

In [8]:
# Alternatively
from operator import add
date_count = day_RDD.map(lambda day: (day, 1)).reduceByKey(add)

In [9]:
# sorting the results
from operator import itemgetter
sorted_items = sorted(date_count.collect(), key=itemgetter(1),reverse=True)

In [10]:
# printing out the results
max_len = max([len(day) for day, count in sorted_items])
for day, count in sorted_items:
    print(f'{day:>{max_len}}: {count}')

2017-02-28: 62103
2017-03-01: 33423
2017-03-02: 32403
2017-03-04: 30225
2017-03-03: 29727
2017-03-08: 29343
2017-03-09: 27288
2017-03-17: 26922
2017-03-07: 26625
2017-03-05: 26019
2017-03-12: 25089
2017-03-13: 24783
2017-03-06: 24315
2017-03-23: 24213
2017-03-14: 23418
2017-03-28: 22338
2017-03-19: 21999
2017-03-20: 21942
2017-03-21: 19410
2017-03-11: 19059
2017-03-18: 17427
2017-03-27: 14544
2017-03-16: 13869
2017-03-26: 13587
2017-03-24: 13467
2017-03-10: 12483
2017-03-25: 12225
2017-03-29: 12120
2017-03-15: 11901
2017-03-22: 10989
2017-03-30: 5814
2017-03-31: 3393
2017-04-01: 537


### Exercise Two: Count the number of measures for each pair of RoomId-SensorId.

In [11]:
# viewing first few elements of the RDD
sensorRDD.take(5)

['2017-03-31 03:38:16.508 1-0 122.153 2.03397',
 '2017-03-31 03:38:15.967 1-1 -3.91901 2.09397',
 '2017-03-31 03:38:16.577 1-2 11.04 2.07397',
 '2017-02-28 00:59:16.359 1-0 19.9884 2.74964',
 '2017-02-28 00:59:16.803 1-1 37.0933 2.76964']

In [12]:
#Splitting each line and extracting only the roomID-sensorID column
id_RDD = sensorRDD.map(lambda line: line.split()[2])
id_RDD.take(10)

['1-0', '1-1', '1-2', '1-0', '1-1', '1-2', '1-0', '1-1', '1-2', '1-0']

In [13]:
#count number how many times each date appears in day_RDD
id_RDD.countByValue()

defaultdict(int,
            {'1-0': 43047,
             '1-1': 43047,
             '1-2': 43047,
             '2-0': 46915,
             '2-1': 46915,
             '2-2': 46915,
             '3-0': 46634,
             '3-1': 46634,
             '3-2': 46634,
             '4-0': 43793,
             '4-1': 43793,
             '4-2': 43793,
             '5-0': 35,
             '5-1': 35,
             '5-2': 35,
             '6-0': 35666,
             '6-1': 35666,
             '6-2': 35666,
             '7-0': 14910,
             '7-1': 14910,
             '7-2': 14910})

### Exercise 3: Compute the average of Value1.

In [14]:
#Splitting each line and extracting only the roomID-sensorID column
value1_RDD = sensorRDD.map(lambda line: line.split()[3])
value1_RDD.take(10)

['122.153',
 '-3.91901',
 '11.04',
 '19.9884',
 '37.0933',
 '45.08',
 '19.3024',
 '38.4629',
 '45.08',
 '19.1652']

In [15]:
#computing the average
value1_RDD.map(lambda x: float(x)).mean()


92.8069927576456

In [42]:
# stopping sonsor spark context
sc.stop()

## 2. Movielens movie data exercises

Movielens (https://movielens.org/) is a website that provides non-commercial, personalised movie recommendations. GroupLens Research has collected and made available rating data sets from the MovieLens web site for the purpose of research into making recommendation services. In this exercise, we will use one of these datasets (the movielens latest dataset, http://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and compute some basic queries on it.
The dataset has already been downloaded and is available at data/movielens/movies.csv, data/movielens/ratings.csv, data/movielens/tags.csv, data/movielens/links.csv

1. Inspect the dataset's [README file](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html), in particular the section titled "Content and Use of Files" to learn the structure of these three files.
2. Compute all pairs (`movieid`, `rat`) where `movieid` is a movie id (as found in ratings.csv) and `rat` is the average rating of that movie id. (Hint: use aggregateByKey to compute first the sum of all ratings as well as the number of ratings per key).
2. Compute all pairs (`title`, `rat`) where `title` is a full movie title (as found in the movies.csv file), and `rat` is the average rating of that movie (computed over all possible ratings for that movie, as found in the ratings.csv file)
3. [_Extra_] Compute all pairs (`title`, `tag`) where `title` is a full movie title that has an average rating of at least 3.5, and `tag` is a tag for that movie (as found in the tags.csv file)

Extra: if you want to experiment with larger datasets, download the 10m dataset (http://files.grouplens.org/datasets/movielens/ml-10m.zip, 250 Mb uncompressed) and re-do the exercises above

In [43]:
from pyspark import SparkConf
# Starting spark and creating a new session
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass

config = SparkConf().setAppName('movie')\
                    .setMaster('local[*]')

In [44]:
from pyspark import SparkContext
sc = SparkContext(conf=config)

#### Average Rating per movie ID.

In [63]:
#reading the rating file as RDD with just the movieid and rating value column.
movie_RDD = sc.textFile('ratings.csv').map(lambda line: (line.split(',')[1], float(line.split(',')[2])))
movie_RDD.take(5)

[('1', '4.0'), ('3', '4.0'), ('6', '4.0'), ('47', '5.0'), ('50', '5.0')]

In [53]:
rating_sum_RDD = movie_RDD.groupByKey().mapValues(sum)
movie_num_RDD = movie_RDD.groupByKey().mapValues(len)


In [57]:
avg_rating_RDD = rating_sum_RDD.join(movie_num_RDD).mapValues(lambda x: (x[0]/x[1]))
avg_rating_RDD.take(5)

[('1', 3.9209302325581397),
 ('50', 4.237745098039215),
 ('70', 3.5090909090909093),
 ('110', 4.031645569620253),
 ('157', 2.8636363636363638)]

#### Average rating per movie title

In [65]:
movie_RDD2 = sc.textFile('movies.csv').map(lambda line: (line.split(',')[0], (line.split(',')[1])))
movie_RDD2.take(5)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)'),
 ('4', 'Waiting to Exhale (1995)'),
 ('5', 'Father of the Bride Part II (1995)')]

In [69]:
result = movie_RDD2.join(avg_rating_RDD).map(lambda x: x[1])
result.take(5)

[('Waiting to Exhale (1995)', 2.357142857142857),
 ('GoldenEye (1995)', 3.496212121212121),
 ('Dracula: Dead and Loving It (1995)', 2.4210526315789473),
 ('Casino (1995)', 3.926829268292683),
 ('Money Train (1995)', 2.5)]

#### Show movie titles and tags only for movies with atleast 3.5 rating

In [70]:
movie_RDD3 = sc.textFile('tags.csv').map(lambda line: (line.split(',')[1], (line.split(',')[2])))
movie_RDD3.take(5)

[('60756', 'funny'),
 ('60756', 'Highly quotable'),
 ('60756', 'will ferrell'),
 ('89774', 'Boxing story'),
 ('89774', 'MMA')]

In [73]:
# movies with alteast 3.5 rating
movie_RDD4 = avg_rating_RDD.filter(lambda x: x[1]>=3.5)
movie_RDD4.take(5)

[('1', 3.9209302325581397),
 ('50', 4.237745098039215),
 ('70', 3.5090909090909093),
 ('110', 4.031645569620253),
 ('163', 3.5606060606060606)]

In [79]:
result2 = movie_RDD2.join(movie_RDD4.join(movie_RDD3)).map(lambda x: x[1])
result2.take(5)

[('Casino (1995)', (3.926829268292683, 'Mafia')),
 ('Babe (1995)', (3.65234375, 'Animal movie')),
 ('Babe (1995)', (3.65234375, 'pigs')),
 ('Babe (1995)',
  (3.65234375, 'villain nonexistent or not needed for good story')),
 ('"Cry', (4.25, 'In Netflix queue'))]

## 3. Github log data exercises
Github makes activity logs publicly available at https://www.githubarchive.org/. One such log file, which contains activity data for 2015-03-01 between 0h-1h at night, has been downloaded and is available at `data/github/2015-03-01-0.json.gz`. This (compressed) file contains multiple JSON objects, one per line. Here is a sample line of this file, neatly formatted:

`{ "id": "2614896652",
    "type": "CreateEvent",
    "actor": {
        "id": 739622,
        "login": "treydock",
        "gravatar_id": "",
        "url": "https://api.githb.com/users/treydock",
        "avatar_url": "https://avatars.githubusercontent.com/u/739622?"
    },
    "repo": {
        "id": 23934080,
        "name": "Early-Modern-OCR/emop-dashboard",
    "url": "https://api.github.com/repos/Early-Modern-OCR/emop-dashboard"
    },
    "payload": {
        "ref": "development",
        "ref_type": "branch",
        "master-branch": "master",
        "description": "",
        "pusher_type": "user",
    },
    "public": true,
    "created_at": "2015-03-01T00:00:00Z",
    "org": {
        "id": 10965476,
        "login": "Early-Modern-OCR",
        "gravatar_id": "",
        "url": "https://api.github.com/orgs/Early-Modern-OCR",
        "avatar_url": "https://avatars.githubusercontent.com/u/10965476?"
    }
}`

This log entry has `CreateEvent` type and its `payload.ref_type` is `branch` . So someone named "treydock" (`actor.login`) created a repository branch called "development" (`payload.ref`) in the first second of March 1, 2015 (`created_at`) .

1. Load the textfile into an RDD (note: spark can read gzipped files directly!). Convert this RDD (which consists of string elements) to an RDD where each element is a JSON object (hint: use the `json.loads` function from the `json` module to convert a string into a JSON object).

2. Filter this RDD of JSON objects to retain only those objects that represent push activities (where `type` equals `PushEvent`)

3. Count the number of push events.

4. Compute the number of push events, grouped per `actor.login`. 

5. Retrieve the results of (4) in sorted order, where logins with higher number of pushes come first. Retrieve the 10 first such results (which contain the highest number of pushes)

6. You are representing a company and need to retrieving the number of pushes for every employee in the company. The file `data/github/employees.txt` contains a list of all employee login names at your company.

Extra: if you want to experiment with larger datasets, download more log data from the github archive website and re-do the exercises above